# Brain-Based Predictive Modeling | Neuroanatomy x Impulsivity

This Jupyter Notebook outlines the brain-based predictive modeling framework when using sMRI data to predict impulsivity-related behaviors in the ABCD data. 

Specific sections need to be customized for different analyses. Details can be found within each of the individual sections and cells.

Elvisha Dhamala, 2024. 

## Import Libraries and Packages

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import sys; sys.path

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, GroupKFold, GroupShuffleSplit, StratifiedKFold
from datetime import datetime


## Specify Subgroup and Time Point

In [ ]:
#specify sex
pred_group = 'm'
group_var=1

#pred_group = all if including all participants
#demo_sex_v2 for sex, 1=M, 2=F, 3=Intersex-M, 4=Intersex-F; 999=Don't know; 777=Refuse to answer
#pred_group = m or f for sex
#race_ethnicity for race/ethnciity, 1 = White; 2 = Black; 3 = Hispanic; 4 = Asian; 5 = Other
#pred_group = wh, bl, hi, as, ot for race/ethnicity

#set the time point you want to consider in your analyses
timepoint = 'baseline_year_1_arm_1'
#for baseline: 'baseline_year_1_arm_1'
#for 2y follow-up: '2_year_follow_up_y_arm_1'
#for 4y follow-up: '4_year_follow_up_y_arm_1'
#for 6y follow-up: '6_year_follow_up_y_arm_1'
pred_yr = '0y'

## Load Imaging Data and Generic ABCD Data (Site, Demo)

In [ ]:
# read and clean up ABCD data
# set base directories
base_dir   = '/Users/elvishadhamala/Documents/BPM/abcd-data-release-5.1/'
BPM_base_dir = '/Users/elvishadhamala/Documents/BPM/'

#load general subj site, qc, and demo data
lt = pd.read_csv(os.path.join(base_dir, 'core/abcd-general/abcd_y_lt.csv'), header=0)
qc = pd.read_csv(os.path.join(base_dir, 'core/imaging/mri_y_qc_incl.csv'), header=0)
clfind = pd.read_csv(os.path.join(base_dir, 'core/imaging/mri_y_qc_clfind.csv'), header=0)
demo = pd.read_csv(os.path.join(base_dir, 'core/abcd-general/abcd_p_demo.csv'), header=0)
icv = pd.read_csv(os.path.join(base_dir, 'core/imaging/mri_y_smr_vol_aseg.csv'), header=0)

#load brain and behavior data
ct = pd.read_csv(os.path.join(base_dir, 'core/imaging/mri_y_smr_thk_dsk.csv'), header=0) 
sa = pd.read_csv(os.path.join(base_dir, 'core/imaging/mri_y_smr_area_dsk.csv'), header=0) 
vol = pd.read_csv(os.path.join(base_dir, 'core/imaging/mri_y_smr_vol_dsk.csv'), header=0) 

#only keep relevant data for qc, site, demo
qc = qc[['src_subject_id', 'eventname', 'imgincl_t1w_include']]
icv = icv[['src_subject_id', 'eventname', 'smri_vol_scs_intracranialv']]
site = lt[['src_subject_id', 'eventname', 'site_id_l']]
demo = demo[['src_subject_id', 'eventname', 'demo_sex_v2', 'race_ethnicity']]
fam = lt[['src_subject_id', 'eventname', 'rel_family_id']]

#demo and family data was collected at baseline so just consider that
demo = demo[demo.eventname == 'baseline_year_1_arm_1']
fam = fam[fam.eventname == 'baseline_year_1_arm_1']


#get rid of participants wtih poor quality img data
qc = qc[qc.imgincl_t1w_include == 1]
clfind = clfind[clfind.mrif_score < 3]

#remove mean columns from ct, sa, and vol
ct = ct.drop(columns=['smri_thick_cdk_meanlh', 'smri_thick_cdk_meanrh', 'smri_thick_cdk_mean'])
sa = sa.drop(columns=['smri_area_cdk_totallh', 'smri_area_cdk_totalrh', 'smri_area_cdk_total'])
vol = vol.drop(columns=['smri_vol_cdk_totallh', 'smri_vol_cdk_totalrh', 'smri_vol_cdk_total'])


#sort data just to make sure they're all in the same order
qc = qc.sort_values(by='src_subject_id', ascending=True)
icv = icv.sort_values(by='src_subject_id', ascending=True)
ct = ct.sort_values(by='src_subject_id', ascending=True)
sa = sa.sort_values(by='src_subject_id', ascending=True)
vol = vol.sort_values(by='src_subject_id', ascending=True)
site = site.sort_values(by='src_subject_id', ascending=True)
fam = fam.sort_values(by='src_subject_id', ascending=True)
clfind = clfind.sort_values(by='src_subject_id', ascending=True)
demo = demo.sort_values(by='src_subject_id', ascending=True)

## Load Behavioral Data

In [ ]:
#specify the behavioral data to be used for the analyses and clean it up based on the variables needed
beh1 = pd.read_csv(os.path.join(base_dir, 'core/mental-health/mh_y_bisbas.csv'), header=0)
beh1 = beh1[['src_subject_id', 'eventname', 
           'bis_y_ss_bis_sum', 
           'bis_y_ss_bas_rr', 'bis_y_ss_bas_drive', 'bis_y_ss_bas_fs']]
beh1 = beh1.dropna(axis='rows')
beh1 = beh1.sort_values(by='src_subject_id', ascending=True)

beh2 = pd.read_csv(os.path.join(base_dir, 'core/mental-health/mh_y_upps.csv'), header=0)
beh2 = beh2[['src_subject_id', 'eventname', 
           'upps_y_ss_negative_urgency', 'upps_y_ss_lack_of_planning', 'upps_y_ss_sensation_seeking',
           'upps_y_ss_positive_urgency', 'upps_y_ss_lack_of_perseverance']]
beh2 = beh2.dropna(axis='rows')
beh2 = beh2.sort_values(by='src_subject_id', ascending=True)

#set name of behavioral prediction domain and specify where to store the results data
results_dir   = '/Users/elvishadhamala/Documents/BPM/preds/preds_bisbas/'
beh_var = 'bisbas'

## Specify the Time Point(s) to be Used

In [ ]:
#get data from specific time point
qc = qc[qc.eventname == timepoint]
icv = icv[icv.eventname == timepoint]
cl = clfind[clfind.eventname == timepoint]
ct = ct[ct.eventname == timepoint]
sa = sa[sa.eventname == timepoint]
vol = vol[vol.eventname == timepoint]
site = site[site.eventname == timepoint]
beh1 = beh1[beh1.eventname == timepoint]
beh2 = beh2[beh2.eventname == timepoint]

## Isolate Participants with Complete Brain and Behavioral Data

In [ ]:
#figure out subjs with complete data across all brain and behavioral variables of interest
series1 = pd.Series(qc.src_subject_id)
series2 = pd.Series(icv.src_subject_id)
series3 = pd.Series(cl.src_subject_id)
series4 = pd.Series(ct.src_subject_id)
series5 = pd.Series(sa.src_subject_id)
series6 = pd.Series(vol.src_subject_id)
series7 = pd.Series(beh1.src_subject_id)
series8 = pd.Series(beh2.src_subject_id)
series9 = pd.Series(demo.src_subject_id)


subj_complete = (set(series1) & set(series2) & set(series3) & set(series4) & 
                 set(series5) & set(series6) & set(series7) & set(series8) & set(series9))
subj_complete = pd.DataFrame(subj_complete)
subj_complete.columns = ['s']

fam = fam[fam.src_subject_id.isin(subj_complete.s)]
fam_unrelated = fam.drop_duplicates(subset=['rel_family_id'], keep='first')
subj_complete_unrelated = pd.DataFrame(fam_unrelated.src_subject_id)
subj_complete_unrelated.columns = ['s']


#isolate data just from participants of interest (i.e., participants with all data)
icv = icv[icv.src_subject_id.isin(subj_complete_unrelated.s)]
ct = ct[ct.src_subject_id.isin(subj_complete_unrelated.s)]
sa = sa[sa.src_subject_id.isin(subj_complete_unrelated.s)]
vol = vol[vol.src_subject_id.isin(subj_complete_unrelated.s)]
site = site[site.src_subject_id.isin(subj_complete_unrelated.s)]
demo = demo[demo.src_subject_id.isin(subj_complete_unrelated.s)]
beh1 = beh1[beh1.src_subject_id.isin(subj_complete_unrelated.s)]
beh2 = beh2[beh2.src_subject_id.isin(subj_complete_unrelated.s)]

## Drop Irrelevant Columns from Data

In [ ]:
#remove subj id and eventname columns, change to array
ct = ct.drop(columns=['src_subject_id', 'eventname'])
sa = sa.drop(columns=['src_subject_id', 'eventname'])
vol = vol.drop(columns=['src_subject_id', 'eventname'])
site = site.drop(columns=['src_subject_id', 'eventname'])
icv = icv.drop(columns=['src_subject_id', 'eventname'])
beh1 = beh1.drop(columns=['src_subject_id', 'eventname'])
beh2 = beh2.drop(columns=['src_subject_id', 'eventname'])

#use specifivc behavioral variable of interest
beh = beh1

#store a list of vars that are in the beh data that will be predicted
beh_names = beh.columns.values

## Apply Mask to Data As Needed

In [ ]:
#create mask based on whatever population subgroup you're looking to model 
mask = demo.demo_sex_v2 == group_var
#mask = ~np.isnan(demo.demo_sex_v2)

ct = ct[mask.values].values
sa = sa[mask.values].values
vol = vol[mask.values].values
beh = beh[mask.values].values
site = site[mask.values].values
icv = icv[mask.values].values

#apply proportional correction to icv data
sa = sa/icv
vol = vol/icv

## Set Up and Run The Predictive Models | Cortical Thickness

In [ ]:
#start with cortical thickness data
X = ct
brain_var = 'ct'

#set y data to be beh
Y = beh

#set site data to be site info
site = site

In [ ]:
#number of repetitions you want to perform
rep = 100

#number of folds you want in the cross-validation
k = 3

#proportion of data you want in your training set and test set
train_size = .80
test_size = 1-train_size

#regression model type
regr = Ridge(normalize=True, max_iter=1000000, solver='lsqr')

#set hyperparameter grid space you want to search through for the model
#adapted from CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
alphas = [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
          3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100]

#param grid set to the hyperparamters you want to search through
paramGrid ={'alpha': alphas}

#number of features 
n_feat = X.shape[1]

#number of behaviors to predict
n_beh = Y.shape[1]


#create variables to store the results
#r^2 - coefficient of determination
r2 = np.zeros([n_beh, rep])
#explained variance
var = np.zeros([n_beh, rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([n_beh, rep])
#optimised alpha (hyperparameter)
opt_alpha = np.zeros([n_beh, rep])
#feature importance extracted from the model
featimp = np.zeros([n_beh, rep, n_feat])
#for when the feat weights get haufe-inverted
featimp_haufe = np.zeros([n_beh, rep, n_feat])


In [ ]:
for b in range(n_beh):
    beh_data = Y[:,b]
    print(beh_names[b])
    
    #iterate through number of repetitions
    for p in range(rep):
    
        #split into train/test data
        train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state=p).split(X, groups=site))
    
        #set x values based on indices from split
        x_train = X[train_inds]
        x_test = X[test_inds]
        
        #set y values based on indices from split  
        beh_train = beh_data[:][train_inds]
        beh_test = beh_data[:][test_inds]
    
        #set site values based on indices from split
        site_train = site[train_inds]
        site_test = site[test_inds] 
    
        #convert y values to to double
        y_train = np.double(beh_train)
        y_test = np.double(beh_test)

        #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
        best_scores = []
        best_params = []

        #set parameters for inner and outer loops for CV
        cv_split = GroupKFold(n_splits=k)


        #define regressor with grid-search CV for inner loop
        gridSearch = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, 
                                  verbose=0, cv=cv_split, scoring='explained_variance')
    
        #fit regressor to the model, use site ID as group category again
        gridSearch.fit(x_train, y_train, groups=site_train)

        #save parameters corresponding to the best score
        best_params.append(list(gridSearch.best_params_.values()))
        best_scores.append(gridSearch.best_score_)

        #save optimised alpha values
        opt_alpha[b,p] = best_params[best_scores.index(np.max(best_scores))][0]

        #rand_alpha = np.random.choice(alphas)

        #fit optimized model
        model = Ridge(alpha = opt_alpha[b,p], normalize=True, max_iter=1000000, solver='lsqr')
        model.fit(x_train, y_train);

        #evaluate model within sex within behavior
        r2[b,p]=model.score(x_test,y_test)

        preds = []
        preds = model.predict(x_test).ravel()

        #compute explained variance 
        var[b,p] = explained_variance_score(y_test, preds)

        #compute correlation between true and predicted (prediction accuracy)
        corr[b,p] = np.corrcoef(y_test.ravel(), preds)[1,0]

        #haufe-transform feature weights
        cov_x = []
        cov_y = []
        y_train_preds = []
        y_train_preds = model.predict(x_train).ravel()

        #extract feature importance
        featimp[b,p,:] = model.coef_
        #compute Haufe-inverted feature weights\
        #cov x is the covariance of the x_train data - covariance of brain data for the training set
        cov_x = np.cov(x_train.T)
        #cov y is the covariance of the y_train predicted data - covariance of the predicted behavior for the training set
        cov_y = np.cov(y_train_preds)
        #this is from eqn 6 of the haufe 2014 paper
        featimp_haufe[b,p,:] = np.matmul(cov_x,featimp[b,p,:])*(1/cov_y)


    #save results
    #np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_r2.npy'),r2)
    #np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_var.npy'),var)
    #np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_corr.npy'),corr)
    #np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_optalpha.npy'),opt_alpha)
    #np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_featimp.npy'),featimp)
    #np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_featimphaufe.npy'),featimp_haufe)


    

## Set Up and Run the Null Models | Cortical Thickness

In [ ]:
#most params stay the same as the tru models, but # of reps increases
rep = 1000

#create variables to store the results
#r^2 - coefficient of determination
r2 = np.zeros([n_beh, rep])
#explained variance
var = np.zeros([n_beh, rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([n_beh, rep])

In [ ]:
for b in range(n_beh):
    #shuffle the data within the sites
    #need to create a new array to store those values so they don't just overwrite the previous one
    values = list(Y)
    values = np.array(values)
    groups = site

    for index in np.unique(groups):
        mask = groups==index
        mask = mask.reshape(-1)
        values[mask] = np.random.permutation(values[mask,:])
    Y_shuffle = values
    beh_data = Y_shuffle[:,b]
    print(beh_names[b])
    
    #iterate through number of models
    for p in range(rep):

        #split into train/test data
        train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state=p).split(X, groups=site))

        #set x values based on indices from split
        x_train = X[train_inds]
        x_test = X[test_inds]
        
        #set y values based on indices from split  
        beh_train = beh_data[:][train_inds]
        beh_test = beh_data[:][test_inds]
    
        site_train = site[train_inds]
        site_test = site[test_inds] 
    
        #convert y values to to double
        y_train = np.double(beh_train)
        y_test = np.double(beh_test)
    
        #randomly choose an alpha value based on the optimized alphas
        rand_alpha = np.random.choice(opt_alpha[b,:])

        #fit optimized models
        model = Ridge(alpha = rand_alpha, normalize=True, max_iter=1000000, solver='lsqr')
        model.fit(x_train, y_train);

        #evaluate model within sex within behavior
        r2[b,p]=model.score(x_test,y_test)

        preds = []
        preds = model.predict(x_test).ravel()

        #compute explained variance 
        var[b,p] = explained_variance_score(y_test, preds)

        #compute correlation between true and predicted (prediction accuracy)
        corr[b,p] = np.corrcoef(y_test.ravel(), preds)[1,0]

    #save results   
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_r2_null.npy'),r2)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_var_null.npy'),var)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_corr_null.npy'),corr)


## Set Up and Run The Predictive Models | Surface Area

In [ ]:
#start with cortical thickness data
X = sa
brain_var = 'sa'

#set y data to be beh
Y = beh

#set site data to be site info
site = site

In [ ]:
#number of repetitions you want to perform
rep = 100

#number of folds you want in the cross-validation
k = 3

#proportion of data you want in your training set and test set
train_size = .80
test_size = 1-train_size

#regression model type
regr = Ridge(normalize=True, max_iter=1000000, solver='lsqr')

#set hyperparameter grid space you want to search through for the model
#adapted from CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
alphas = [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
          3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100]

#param grid set to the hyperparamters you want to search through
paramGrid ={'alpha': alphas}

#number of features 
n_feat = X.shape[1]

#number of behaviors to predict
n_beh = Y.shape[1]


#create variables to store the results
#r^2 - coefficient of determination
r2 = np.zeros([n_beh, rep])
#explained variance
var = np.zeros([n_beh, rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([n_beh, rep])
#optimised alpha (hyperparameter)
opt_alpha = np.zeros([n_beh, rep])
#feature importance extracted from the model
featimp = np.zeros([n_beh, rep, n_feat])
#for when the feat weights get haufe-inverted
featimp_haufe = np.zeros([n_beh, rep, n_feat])


In [ ]:
for b in range(n_beh):
    beh_data = Y[:,b]
    print(beh_names[b])
    
    #iterate through number of repetitions
    for p in range(rep):
    
        #split into train/test data
        train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state=p).split(X, groups=site))
    
        #set x values based on indices from split
        x_train = X[train_inds]
        x_test = X[test_inds]
        
        #set y values based on indices from split  
        beh_train = beh_data[:][train_inds]
        beh_test = beh_data[:][test_inds]
    
        #set site values based on indices from split
        site_train = site[train_inds]
        site_test = site[test_inds] 
    
        #convert y values to to double
        y_train = np.double(beh_train)
        y_test = np.double(beh_test)

        #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
        best_scores = []
        best_params = []

        #set parameters for inner and outer loops for CV
        cv_split = GroupKFold(n_splits=k)


        #define regressor with grid-search CV for inner loop
        gridSearch = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, verbose=0, cv=cv_split, scoring='explained_variance')
    
        #fit regressor to the model, use site ID as group category again
        gridSearch.fit(x_train, y_train, groups=site_train)

        #save parameters corresponding to the best score
        best_params.append(list(gridSearch.best_params_.values()))
        best_scores.append(gridSearch.best_score_)

        #save optimised alpha values
        opt_alpha[b,p] = best_params[best_scores.index(np.max(best_scores))][0]

        #rand_alpha = np.random.choice(alphas)

        #fit optimized model
        model = Ridge(alpha = opt_alpha[b,p], normalize=True, max_iter=1000000, solver='lsqr')
        model.fit(x_train, y_train);

        #evaluate model within sex within behavior
        r2[b,p]=model.score(x_test,y_test)

        preds = []
        preds = model.predict(x_test).ravel()

        #compute explained variance 
        var[b,p] = explained_variance_score(y_test, preds)

        #compute correlation between true and predicted (prediction accuracy)
        corr[b,p] = np.corrcoef(y_test.ravel(), preds)[1,0]

        #haufe-transform feature weights
        cov_x = []
        cov_y = []
        y_train_preds = []
        y_train_preds = model.predict(x_train).ravel()

        #extract feature importance
        featimp[b,p,:] = model.coef_
        #compute Haufe-inverted feature weights\
        #cov x is the covariance of the x_train data - covariance of brain data for the training set
        cov_x = np.cov(x_train.T)
        #cov y is the covariance of the y_train predicted data - covariance of the predicted behavior for the training set
        cov_y = np.cov(y_train_preds)
        #this is from eqn 6 of the haufe 2014 paper
        featimp_haufe[b,p,:] = np.matmul(cov_x,featimp[b,p,:])*(1/cov_y)


    #save results
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_r2.npy'),r2)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_var.npy'),var)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_corr.npy'),corr)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_optalpha.npy'),opt_alpha)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_featimp.npy'),featimp)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_featimphaufe.npy'),featimp_haufe)


    

## Set Up and Run the Null Models | Surface Area

In [ ]:
#most params stay the same as the tru models, but # of reps increases
rep = 1000

#create variables to store the results
#r^2 - coefficient of determination
r2 = np.zeros([n_beh, rep])
#explained variance
var = np.zeros([n_beh, rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([n_beh, rep])

In [ ]:
for b in range(n_beh):
    #shuffle the data within the sites
    #need to create a new array to store those values so they don't just overwrite the previous one
    values = list(Y)
    values = np.array(values)
    groups = site

    for index in np.unique(groups):
        mask = groups==index
        mask = mask.reshape(-1)
        values[mask] = np.random.permutation(values[mask,:])
    Y_shuffle = values
    beh_data = Y_shuffle[:,b]
    print(beh_names[b])
    
    #iterate through number of models
    for p in range(rep):

        #split into train/test data
        train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state=p).split(X, groups=site))

        #set x values based on indices from split
        x_train = X[train_inds]
        x_test = X[test_inds]
        
        #set y values based on indices from split  
        beh_train = beh_data[:][train_inds]
        beh_test = beh_data[:][test_inds]
    
        site_train = site[train_inds]
        site_test = site[test_inds] 
    
        #convert y values to to double
        y_train = np.double(beh_train)
        y_test = np.double(beh_test)
    
        #randomly choose an alpha value based on the optimized alphas
        rand_alpha = np.random.choice(opt_alpha[b,:])

        #fit optimized models
        model = Ridge(alpha = rand_alpha, normalize=True, max_iter=1000000, solver='lsqr')
        model.fit(x_train, y_train);

        #evaluate model within sex within behavior
        r2[b,p]=model.score(x_test,y_test)

        preds = []
        preds = model.predict(x_test).ravel()

        #compute explained variance 
        var[b,p] = explained_variance_score(y_test, preds)

        #compute correlation between true and predicted (prediction accuracy)
        corr[b,p] = np.corrcoef(y_test.ravel(), preds)[1,0]

    #save results   
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_r2_null.npy'),r2)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_var_null.npy'),var)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_corr_null.npy'),corr)


## Set Up and Run The Predictive Models | Gray Matter Volume

In [ ]:
#start with cortical thickness data
X = vol
brain_var = 'vol'

#set y data to be beh
Y = beh

#set site data to be site info
site = site

In [ ]:
#number of repetitions you want to perform
rep = 100

#number of folds you want in the cross-validation
k = 3

#proportion of data you want in your training set and test set
train_size = .80
test_size = 1-train_size

#regression model type
regr = Ridge(normalize=True, max_iter=1000000, solver='lsqr')

#set hyperparameter grid space you want to search through for the model
#adapted from CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
alphas = [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
          3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100]

#param grid set to the hyperparamters you want to search through
paramGrid ={'alpha': alphas}

#number of features 
n_feat = X.shape[1]

#number of behaviors to predict
n_beh = Y.shape[1]


#create variables to store the results
#r^2 - coefficient of determination
r2 = np.zeros([n_beh, rep])
#explained variance
var = np.zeros([n_beh, rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([n_beh, rep])
#optimised alpha (hyperparameter)
opt_alpha = np.zeros([n_beh, rep])
#feature importance extracted from the model
featimp = np.zeros([n_beh, rep, n_feat])
#for when the feat weights get haufe-inverted
featimp_haufe = np.zeros([n_beh, rep, n_feat])


In [ ]:
for b in range(n_beh):
    beh_data = Y[:,b]
    print(beh_names[b])
    
    #iterate through number of repetitions
    for p in range(rep):
    
        #split into train/test data
        train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state=p).split(X, groups=site))
    
        #set x values based on indices from split
        x_train = X[train_inds]
        x_test = X[test_inds]
        
        #set y values based on indices from split  
        beh_train = beh_data[:][train_inds]
        beh_test = beh_data[:][test_inds]
    
        #set site values based on indices from split
        site_train = site[train_inds]
        site_test = site[test_inds] 
    
        #convert y values to to double
        y_train = np.double(beh_train)
        y_test = np.double(beh_test)

        #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
        best_scores = []
        best_params = []

        #set parameters for inner and outer loops for CV
        cv_split = GroupKFold(n_splits=k)


        #define regressor with grid-search CV for inner loop
        gridSearch = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, verbose=0, cv=cv_split, scoring='explained_variance')
    
        #fit regressor to the model, use site ID as group category again
        gridSearch.fit(x_train, y_train, groups=site_train)

        #save parameters corresponding to the best score
        best_params.append(list(gridSearch.best_params_.values()))
        best_scores.append(gridSearch.best_score_)

        #save optimised alpha values
        opt_alpha[b,p] = best_params[best_scores.index(np.max(best_scores))][0]

        #rand_alpha = np.random.choice(alphas)

        #fit optimized model
        model = Ridge(alpha = opt_alpha[b,p], normalize=True, max_iter=1000000, solver='lsqr')
        model.fit(x_train, y_train);

        #evaluate model within sex within behavior
        r2[b,p]=model.score(x_test,y_test)

        preds = []
        preds = model.predict(x_test).ravel()

        #compute explained variance 
        var[b,p] = explained_variance_score(y_test, preds)

        #compute correlation between true and predicted (prediction accuracy)
        corr[b,p] = np.corrcoef(y_test.ravel(), preds)[1,0]

        #haufe-transform feature weights
        cov_x = []
        cov_y = []
        y_train_preds = []
        y_train_preds = model.predict(x_train).ravel()

        #extract feature importance
        featimp[b,p,:] = model.coef_
        #compute Haufe-inverted feature weights\
        #cov x is the covariance of the x_train data - covariance of brain data for the training set
        cov_x = np.cov(x_train.T)
        #cov y is the covariance of the y_train predicted data - covariance of the predicted behavior for the training set
        cov_y = np.cov(y_train_preds)
        #this is from eqn 6 of the haufe 2014 paper
        featimp_haufe[b,p,:] = np.matmul(cov_x,featimp[b,p,:])*(1/cov_y)


    #save results
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_r2.npy'),r2)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_var.npy'),var)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_corr.npy'),corr)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_optalpha.npy'),opt_alpha)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_featimp.npy'),featimp)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_featimphaufe.npy'),featimp_haufe)


    

## Set Up and Run the Null Models | Gray Matter Volume

In [ ]:
#most params stay the same as the tru models, but # of reps increases
rep = 1000

#create variables to store the results
#r^2 - coefficient of determination
r2 = np.zeros([n_beh, rep])
#explained variance
var = np.zeros([n_beh, rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([n_beh, rep])

In [ ]:
for b in range(n_beh):
    #shuffle the data within the sites
    #need to create a new array to store those values so they don't just overwrite the previous one
    values = list(Y)
    values = np.array(values)
    groups = site

    for index in np.unique(groups):
        mask = groups==index
        mask = mask.reshape(-1)
        values[mask] = np.random.permutation(values[mask,:])
    Y_shuffle = values
    beh_data = Y_shuffle[:,b]
    print(beh_names[b])
    
    #iterate through number of models
    for p in range(rep):

        #split into train/test data
        train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state=p).split(X, groups=site))

        #set x values based on indices from split
        x_train = X[train_inds]
        x_test = X[test_inds]
        
        #set y values based on indices from split  
        beh_train = beh_data[:][train_inds]
        beh_test = beh_data[:][test_inds]
    
        site_train = site[train_inds]
        site_test = site[test_inds] 
    
        #convert y values to to double
        y_train = np.double(beh_train)
        y_test = np.double(beh_test)
    
        #randomly choose an alpha value based on the optimized alphas
        rand_alpha = np.random.choice(opt_alpha[b,:])

        #fit optimized models
        model = Ridge(alpha = rand_alpha, normalize=True, max_iter=1000000, solver='lsqr')
        model.fit(x_train, y_train);

        #evaluate model within sex within behavior
        r2[b,p]=model.score(x_test,y_test)

        preds = []
        preds = model.predict(x_test).ravel()

        #compute explained variance 
        var[b,p] = explained_variance_score(y_test, preds)

        #compute correlation between true and predicted (prediction accuracy)
        corr[b,p] = np.corrcoef(y_test.ravel(), preds)[1,0]

    #save results   
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_r2_null.npy'),r2)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_var_null.npy'),var)
    np.save((results_dir + brain_var + '_' + beh_var + '_' + pred_group + '_' + pred_yr + '_corr_null.npy'),corr)
